In [1]:
import os
import sys
import torch
import numpy as np
from torch.distributions import MultivariateNormal
from scipy.stats import invwishart
from numpy.linalg import inv, det
from torch import matmul as m
from torch import einsum

from deep_fields.models.random_fields.blocks_utils_torch import calculate_determinant_and_inverse_covariance_history_torch
from deep_fields.models.random_fields.blocks_utils_torch import obtain_location_index_to_realization, obtain_blocks
from deep_fields.models.random_fields.blocks_utils_torch import log_probability_from_blocks

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
total_assets_in_history = 30
birth_numbers = [5,1,2,2,2,4,2,2,5,5]
birth_numbers = np.array(birth_numbers)
assets_in_the_market = birth_numbers.cumsum()
number_of_realizations = len(assets_in_the_market)
location_index_to_realization = obtain_location_index_to_realization(birth_numbers)

In [3]:
#generate covariance matrix prior
nu = total_assets_in_history + 1.
Psi = np.random.rand(total_assets_in_history, total_assets_in_history)
Psi = np.dot(Psi, Psi.transpose())

a_J = np.ones(total_assets_in_history)*10.
b_J = 1.

lambda_ = 1/b_J
mu_0 = a_J

IW = invwishart(nu,Psi)
covariance_matrix = torch.Tensor(IW.rvs())
mu = torch.Tensor(np.ones(total_assets_in_history)*0.01)
covariance_matrix = covariance_matrix.type(torch.float64)
mu = mu.type(torch.float64)

In [4]:
number_of_realizations

10

In [5]:
# generate data
distribution = MultivariateNormal(mu,covariance_matrix)
sample = distribution.sample(sample_shape=(number_of_realizations,))
for i,current_number_of_assets in enumerate(assets_in_the_market):
    sample[i,current_number_of_assets:] = torch.zeros_like(sample[i,current_number_of_assets:])
log_returns = sample

In [6]:
# CALCULATES DETERMINANT AND COVARIANCE
determinants_history, inverse_covariance_history = calculate_determinant_and_inverse_covariance_history_torch(
    assets_in_the_market, covariance_matrix, False)

In [9]:
log_probability = log_probability_from_blocks(sample, mu, assets_in_the_market, determinants_history, inverse_covariance_history)

In [13]:
# OLD CALCULATION (EXACT; LOG PROB CALCULATED AT EACH STEP)

exact_determinants = []
exact_probability = []
exact_bilinear = []

for time_index in range(number_of_realizations):
    current_assets_in_the_market = assets_in_the_market[time_index]
    current_interest_rate = mu[None, :current_assets_in_the_market]
    current_diffusion_covariance = covariance_matrix[:current_assets_in_the_market, :current_assets_in_the_market]
    
    exact_determinants.append(torch.det(current_diffusion_covariance).type(torch.float64))
    
    current_inverse_covariance = torch.inverse(current_diffusion_covariance)
    vector =  sample[time_index][:current_assets_in_the_market] - current_interest_rate
    
    exact_inverse_covariance = einsum("bi,bij,bj->b",vector,current_inverse_covariance[None,:,:],vector).item()
    exact_bilinear.append(exact_inverse_covariance)
    
    current_diffusion_distribution = MultivariateNormal(current_interest_rate,
                                                        current_diffusion_covariance)
    
    current_log_returns = sample[time_index, :current_assets_in_the_market]
    cdd = current_diffusion_distribution.log_prob(torch.Tensor(current_log_returns))
    
    exact_probability.append(cdd.item())

TypeError: expected Float (got Double)

In [149]:
exact_determinants

[36214.56857289441,
 5847.061102570047,
 327.8112366336968,
 8.468830052459685,
 0.2563999758606705,
 0.00012570311644313268,
 2.051852188257407e-07,
 6.174471064621544e-10,
 8.110634670427716e-18,
 6.6766975324361495e-28]

In [150]:
determinants_history

array([3.62145686e+04, 5.84706110e+03, 3.27811306e+02, 8.46882024e+00,
       2.56392836e-01, 1.25694572e-04, 2.05114934e-07, 6.17761746e-10,
       8.18235929e-18, 7.06196521e-28])

In [151]:
exact_bilinear

[5.182182312011719,
 5.027256011962891,
 3.903975486755371,
 6.9319000244140625,
 12.390270233154297,
 8.763862609863281,
 12.73712158203125,
 14.716777801513672,
 18.141910552978516,
 39.0933837890625]

In [152]:
log_probability_determinant

array([19.6866021 , 19.70095684, 20.49545469, 20.51516188, 20.6934803 ,
       20.42437744, 17.68209183, 15.55262307,  6.60237552, -7.38134724])

In [153]:
exact_log_probability_determinant = np.log(((2*np.pi)**assets_in_the_market)*np.asarray(exact_determinants))
exact_log_probability_determinant

array([19.6866021 , 19.70095684, 20.49545448, 20.51516303, 20.69350815,
       20.42444541, 17.68243444, 15.55211362,  6.59357111, -7.43744713])

In [154]:
-.5*(exact_log_probability_determinant + np.asarray(exact_bilinear))

array([-12.4343922 , -12.36410642, -12.19971498, -13.72353153,
       -16.54188919, -14.59415401, -15.20977801, -15.13444571,
       -12.36774083, -15.82796833])

In [155]:
log_probability

array([-12.43457912, -12.36411978, -12.19967885, -13.72443503,
       -16.53726521, -14.59544516, -15.20591789, -15.13508954,
       -12.34190774, -15.3843691 ])

In [156]:
np.asarray(exact_probability)

array([-12.43465996, -12.36403084, -12.19915199, -13.72570133,
       -16.54165268, -14.5979805 , -15.2044754 , -15.13560104,
       -12.34396172, -15.64263916])

# Blocks Inversion and Determinant

In [88]:
def log_probability_of_growth(result,determinant):
    """
    """
    log_probability = np.log(((2*np.pi)**total_assets_in_history)*determinant)+np.dot(vector,np.dot(result,vector))
    log_probability = -.5*log_probability
    return log_probability